<h1><strong> 대기오염 및 기상 데이터 전처리 </strong></h1>
<br>

<hr>

<br>
<h2> 1. Introduction </h2>
 <p style="font-size:16px">📖 개요 : 본 노트북은 2023년부터 2025년까지 3년의 서울시 대기오염 데이터와 기상 데이터를 머신러닝 모델링에 사용하기 적합한 형태로 정제하는 전처리 과정을 기록한 리포트이다.
 <p style="font-size:16px">🏁 목표 : 서울시 공공데이터와 기상자료개방포털에서 제공받은 데이터를 로드하고, 필요한 컬럼을 리네임, 결측치 처리, 시계열 정렬 등의 전처리 과정을 수행하며 학습용 정제 데이터셋을 산출한다.</p>

<br>
<hr>
<br>
<h2> 2. Library Import </h2>


In [18]:
import pandas as pd
import numpy as np

<hr>
<br>
<h2> 3. 서울시 대기오염 및 기상 데이터 전처리 함수 </h2>

In [19]:
def load_preprocess_air_pollution(filepath, encoding):
    data_frame = pd.read_csv(filepath, encoding=encoding)
    
    data_frame.rename(columns={
        '측정일시': 'date',
        '측정소명': 'station_name',
        '미세먼지농도(㎍/㎥)': 'PM10',
        '미세먼지농도 (㎍/㎥)': 'PM10',
        '초미세먼지농도(㎍/㎥)': 'PM2.5',
        '초미세먼지농도 (㎍/㎥)': 'PM2.5',
        '오존농도(ppm)': 'O3',
        '이산화질소농도(ppm)': 'NO2',
        '일산화탄소농도(ppm)': 'CO',
        '아황산가스농도(ppm)': 'SO2',
        '아황산가스농도 (ppm)': 'SO2'
    }, inplace=True)

    data_frame['date'] = pd.to_datetime(
        data_frame['date'].astype(str),
        format='%Y%m%d',
        errors='coerce')

    features = ['PM10', 'PM2.5', 'O3', 'NO2', 'CO', 'SO2']
    for col in features:
        data_frame[col] = pd.to_numeric(data_frame[col], errors='coerce')

    data_frame.sort_values('date', inplace=True)
    data_frame[features] = data_frame[features].ffill().bfill()

    train_set = data_frame[['date'] +['station_name'] + features]
    return train_set

<p style="font-size:16px">
   위 함수는 대기 오염 데이터를 불러오고, 분석에 적합한 형태로 전처리를 수행한다.
</p><br>

```
data_frame.rename(columns={...}, inplace=True)
```

<p style="font-size:16px">   
   한국어 컬럼 이름을 영어로 표준화하며 편의성을 향상시켰다.
</p><br>

```
data_frame['date'] = pd.to_datetime(data_frame['date'].astype(str), format='%Y%m%d', errors='coerce')
```

<p style="font-size:16px">
   대기 오염 데이터에서 YYYYMMDD 형태의 date 컬럼을 datetime 타입으로 변환시킨다.
</p><br>

```
data_frame[col] = pd.to_numeric(data_frame[col], errors='coerce')
```

<p style="font-size:16px">
   날짜 컬럼을 제외한 나머지 컬럼들의 데이터를 숫자로 변환하고, 불가한 값이 있다면 NaN처리를 한다.
</p><br>

```
data_frame.sort_values('date', inplace=True)
data_frame[features] = data_frame[features].ffill().bfill()
```

<p style="font-size:16px">
   날짜 순으로 정렬을 한 후, 존재하는 결측치는 앞과 뒤의 값으로 채워주었다.
</p><br>

```
train_set = data_frame[['date', 'station_name'] + features]
```

<p style="font-size:16px">
   최종적으로 날짜와 측정소명과 나머지 컬럼들을 합쳐서 반환한다.
</p>

In [20]:
def load_process_weather(filepath):
    data_frame = pd.read_csv(filepath)

    data_frame.rename(columns={
        '일시': 'date',
        '지점': 'station_id',
        '지점명': 'station_name',
        '평균기온(°C)': 'temp_c_mean',
        '최저기온(°C)': 'temp_c_min',
        '최고기온(°C)': 'temp_c_max',
        '일강수량(mm)': 'precip_mm',
        '평균 풍속(m/s)': 'wind_ms',
        '평균 상대습도(%)': 'rh',
        '평균 해면기압(hPa)': 'mslp'
    }, inplace=True)

    data_frame['date'] = pd.to_datetime(data_frame['date'], errors='coerce')
    data_frame.drop(columns=['station_id', 'station_name'], inplace=True,
                    errors='ignore')
    features = ['temp_c_mean', 'temp_c_min', 'temp_c_max', 'precip_mm', 'wind_ms', 'rh', 'mslp']
    
    for col in features:
        data_frame[col] = pd.to_numeric(data_frame[col], errors='coerce')

    data_frame.sort_values('date', inplace=True)
    data_frame[features] = data_frame[features].ffill().bfill()

    train_set = data_frame[['date'] + features]

    return train_set

<p style="font-size:16px">
   위 함수는 기상 데이터를 불러오고, 분석에 적합한 형태로 전처리를 수행한다.
</p><br>

```
data_frame.rename(columns={...}, inplace=True)
```

<p style="font-size:16px">   
   한국어 컬럼 이름을 영어로 표준화하며 편의성을 향상시켰다.
</p><br>

```
data_frame['date'] = pd.to_datetime(data_frame['date'], errors='coerce')
```

<p style="font-size:16px">
   기상 데이터에서 YYYY-MM-DD 형태의 date 컬럼을 datetime 타입으로 변환시킨다.
</p><br>

```
data_frame.drop(columns=['station_id', 'station_name'], inplace=True, errors='ignore')
```

<p style="font-size:16px">
   사용하지 않을 측정소 번호와 이름 컬럼은 데이터프레임에서 제거한다.
</p><br>

```
data_frame[col] = pd.to_numeric(data_frame[col], errors='coerce')
```

<p style="font-size:16px">
   날짜 컬럼을 제외한 나머지 컬럼들의 데이터를 숫자로 변환하고, 불가한 값이 있다면 NaN처리를 한다.
</p><br>

```
data_frame.sort_values('date', inplace=True)
data_frame[features] = data_frame[features].ffill().bfill()
```

<p style="font-size:16px">
   날짜 순으로 정렬을 한 후, 존재하는 결측치는 앞과 뒤의 값으로 채워주었다.
</p><br>

```
train_set = data_frame[['date', 'station_name'] + features]
```

<p style="font-size:16px">
   최종적으로 날짜와 컬럼들을 합쳐서 반환한다.
</p>
<br>
<hr>

<h2> 4. 함수 실행 & 결과 확인 </h2>

In [ ]:
air_pollution_df_2025 = load_preprocess_air_pollution('../data/air-pollution-info-2025.csv', encoding='utf-8-sig')
air_pollution_df_2024 = load_preprocess_air_pollution('../data/air-pollution-info-2024.csv', encoding='cp949')
air_pollution_df_2023 = load_preprocess_air_pollution('../data/air-pollution-info-2023.csv', encoding='cp949')
weather_df = load_process_weather('../data/weather-info.csv')

air_pollution_df = pd.concat(
    [air_pollution_df_2023, air_pollution_df_2024, air_pollution_df_2025],
    ignore_index=True
)

print("✅ 대기 오염 데이터프레임 출력")
display(air_pollution_df.head(5))

print("✅ 기상 데이터프레임 출력")
display(weather_df.head(5))

✅대기 오염 데이터프레임 출력


,date,station_name,PM10,PM2.5,O3,NO2,CO,SO2
0,2023-01-01,강남구,52.0,36.0,0.024,0.024,0.6,0.004
1,2023-01-01,공항대로,53.0,36.0,0.024,0.018,0.6,0.003
2,2023-01-01,도산대로,54.0,38.0,0.019,0.024,1.0,0.004
3,2023-01-01,동작대로,58.0,37.0,0.023,0.026,0.7,0.004
4,2023-01-01,시흥대로,47.0,29.0,0.020,0.027,0.6,0.004


✅기상 데이터프레임 출력


,date,temp_c_mean,temp_c_min,temp_c_max,precip_mm,wind_ms,rh,mslp
0,2022-07-10,29.2,24.6,35.1,0.0,1.7,68.0,1004.1
1,2022-07-11,27.0,24.7,30.6,9.0,1.8,75.0,1005.5
2,2022-07-12,26.8,23.2,29.4,9.0,2.2,73.1,1007.1
3,2022-07-13,25.0,24.0,27.1,114.5,1.8,93.9,1001.9
4,2022-07-14,25.7,23.3,29.9,0.9,2.3,84.6,999.8


<p style="font-size:16px"> 연도별 데이터셋을 합친 후 최상위 5줄만 출력한 상태이다. 전처리가 예외없이 잘 수행되었다.</p>

In [ ]:
df = pd.merge(air_pollution_df, weather_df, on='date', how='inner')
df = df[df['station_name'] == '강남구']
print("✅ Inner Join 실행 및 강남구 필터링 후")
display(df.head(5))

✅ Inner Join 실행 및 강남구 필터링 후


,date,station_name,PM10,PM2.5,O3,NO2,CO,SO2,temp_c_mean,temp_c_min,temp_c_max,precip_mm,wind_ms,rh,mslp
0,2023-01-01,강남구,52.0,36.0,0.0240,0.0240,0.60,0.0040,-0.2,-4.3,3.8,0.0,2.7,55.0,1030.8
88,2023-01-02,강남구,24.0,11.0,0.0212,0.0206,0.43,0.0034,-4.5,-7.4,-0.4,0.0,2.5,46.0,1032.2
137,2023-01-03,강남구,29.0,15.0,0.0162,0.0303,0.51,0.0034,-5.0,-9.0,0.6,0.0,1.8,49.0,1030.7
176,2023-01-04,강남구,35.0,22.0,0.0111,0.0412,0.61,0.0036,-1.8,-5.7,3.3,0.0,1.9,51.0,1030.5
228,2023-01-05,강남구,47.0,32.0,0.0056,0.0523,0.80,0.0038,-1.6,-5.6,3.6,0.0,1.6,58.1,1030.6


<p style="font-size:16px"> 위의 데이터프레임은 대기 오염 데이터와 기상 데이터를 date 기준으로 <strong><mark>Inner Join<mark></strong>하여 병합한 상태이다. 따라서 해당 날짜의 대기 오염 현황과 기상 상태를 쉽게 파악할 수 있다. <br><br> 서울의 중심지인 '강남구'를 기준으로 선택했기 때문에, 이 외의 station_name을 가진 행은 제거하였다.</p><br><hr>

<h2> 5. 결론 </h2>
<p style="font-size:16px">
    - 대기오염 데이터와 기상 데이터에 대해 <strong>컬럼명 표준화</strong>를 진행하면서 편리성 향상<br>
    - 주요 feature에 대해 ffill().bfill()로 <strong>결측치 처리</strong><br>
    - <strong>Inner Join</strong>과 </strong>데이터셋 병합</strong>을 통해 분석을 위한 데이터프레임 생성<br><br>
    강남구에서의 날짜별 대기 오염 상태와 기상 상태의 정보를 추출한 데이터셋을 만들었으므로, 이 데이터셋을 이용해 본격적으로 로지스틱 회귀를 위한 훈련을 다음 단계에 실행할 예정이다.<br>
    
    -> notebooks/training.ipynb
</p>